# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [8]:
# import libraries
import pandas as pd
import numpy as np


In [9]:
# load messages dataset
messages = pd.read_csv('messages.csv')
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [12]:
#function to read csv from categories.csv inorder to clean data  : 
def read_categories(path_csv='categories.csv'):
    #helper function to manage column list 
    def column_update(l,entry) :
        if entry in l:
            pass
        else:
            l.append(entry)
    #reading the given file 
    fp= open(path_csv,'r')
    columns=[] # list to store the column data
    while(True):
        a=fp.readline()
        if a =='':
            print(columns)
            break
            fp.close()
            yield None
        else:
            a=a.replace('\n','')
            items=a.split(',')
            result=[]
            for item in items:
                splits=item.split(';')
                if item=='categories' :
                    continue
                if item=='id':
                    column_update(columns,item)
                    continue
                elif len(splits) >2 : 
                    for vals in splits:
                        tmp=vals.split('-')
                        column_update(columns,tmp[0])
                        result.append(int(tmp[-1]))
                else:
                    result.append(int(splits[0]))
            if len(result)>=1:
                yield result
            else:
                continue

In [11]:
a=read_categories()
#printing columns for dataframe 
while(next(a)):
    continue

['id', 'related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [13]:
#storing columns data 
columns=['id', 'related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']

In [14]:
# load categories dataset
categories = pd.DataFrame(read_categories(), columns=columns)
categories['id']=categories['id'].apply(lambda x : int(x)) # it will help to merge with the id column in messages dataframe
categories.head()

['id', 'related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,1,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,12,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
print(len(messages))
print(len(categories))

26248
26248


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [16]:
messages=messages[~messages.duplicated()]
categories=categories[~categories.duplicated()]

In [17]:
print(len(messages))
print(len(categories))

26180
26216


In [18]:
set(messages['id'])-set(categories['id'])

set()

In [19]:
set(categories['id'])-set(messages['id'])

set()

In [20]:
categories.isna().any().any()

False

In [21]:
messages.isna().any()

id          False
message     False
original     True
genre       False
dtype: bool

In [22]:
for i in categories[categories['id'].duplicated()]['id']:
    print(categories[categories['id']==i])
    

      id  related  request  offer  aid_related  medical_help  \
162  202        1        1      0            1             0   
163  202        1        1      0            1             0   

     medical_products  search_and_rescue  security  military      ...        \
162                 0                  0         0         0      ...         
163                 0                  0         0         0      ...         

     aid_centers  other_infrastructure  weather_related  floods  storm  fire  \
162            0                     0                0       0      0     0   
163            0                     0                0       0      0     0   

     earthquake  cold  other_weather  direct_report  
162           0     0              0              0  
163           0     0              0              0  

[2 rows x 37 columns]
      id  related  request  offer  aid_related  medical_help  \
709  862        0        0      0            0             0   
710  862       

          id  related  request  offer  aid_related  medical_help  \
16280  19142        1        0      0            1             0   
16281  19142        1        0      0            1             0   

       medical_products  search_and_rescue  security  military      ...        \
16280                 0                  0         0         0      ...         
16281                 0                  0         0         0      ...         

       aid_centers  other_infrastructure  weather_related  floods  storm  \
16280            0                     0                1       0      0   
16281            0                     0                1       0      0   

       fire  earthquake  cold  other_weather  direct_report  
16280     0           0     1              0              0  
16281     0           0     1              0              0  

[2 rows x 37 columns]
          id  related  request  offer  aid_related  medical_help  \
16768  19687        1        0      0        

In [23]:
# merge datasets
df = messages.merge(categories ,on = 'id', how='left')
print(len(df))
df.head()

26216


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
cat_column=['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']

### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [39]:
def series_int(x):
    z=x.apply(lambda a : int(a))
    return z 
    
#categories[cat_column].apply(lambda x : series_int(x) ,axis=1)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
# create a dataframe of the 36 individual category columns
categories = 
categories.head()

In [ ]:
# select the first row of the categories dataframe
row = 

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = 
print(category_colnames)

In [56]:
# rename the columns of `categories`
#categories.columns = category_colnames
categories.head()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,1,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,12,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [ ]:
for column in categories:
    # set each value to be the last character of the string
    categories[column] = 
    
    # convert column from string to numeric
    categories[column] = 
categories.head()

### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [ ]:
# drop the original categories column from `df`


df.head()

In [ ]:
# concatenate the original dataframe with the new `categories` dataframe
df = 
df.head()

### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [ ]:
# check number of duplicates


In [ ]:
# drop duplicates


In [ ]:
# check number of duplicates


### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [57]:
from sqlalchemy import create_engine

In [58]:
engine = create_engine('sqlite:///TrialDatabaseName.db')
df.to_sql('messagelabel', engine, index=False)

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.

In [ ]:
#An example command to run etl process
#python process_data.py disaster_messages.csv disaster_categories.csv DisasterResponse.db

In [1]:
%%writefile etl_pipeline.py
import pandas as pd
import numpy as np
import sys
from sqlalchemy import create_engine


def get_file_names():
    '''Capture file name arguments passed to command line 
    and transfer to relevant functions as their required arguments'''
    files=list()
    for arg in sys.argv:
        files.append(arg)
    if len(files)!=4:
        raise ValueError("Provide three filenames to the pipeline\
        in sequence messages.csv,\
        categories.csv , databasse.db")
    if files[1].split('.')[-1]!='csv' :
        raise ValueError (f"Provide .CSV filename at {files[1]} ")
    if files[2].split('.')[-1]!='csv' :
        raise ValueError (f"Provide .CSV filename at {files[2]} ")
    if files[3].split('.')[-1]!='db' :
        raise ValueError (f"Provide .db extension at {files[3]} ")
    return files

#helper function to manage column list 
def column_update(l,entry) :
    if entry in l:
        pass
    else:
        l.append(entry)

columns=[] # list to store the column data

def read_categories(path_csv):
    '''This function is to read the categories'
    matrix-data from the categories.csv '''
    #reading the given file 
    fp= open(path_csv,'r')
    while(True):
        a=fp.readline()
        if a =='':
            fp.close()
            break
            yield None
        else:
            a=a.replace('\n','')
            items=a.split(',')
            result=[]
            for item in items:
                splits=item.split(';')
                if item=='categories' :
                    continue
                if item=='id':
                    column_update(columns,item)
                    continue
                elif len(splits) >2 : 
                    for vals in splits:
                        tmp=vals.split('-')
                        column_update(columns,tmp[0])
                        result.append(int(tmp[-1]))
                else:
                    result.append(int(splits[0]))
            if len(result)>=1:
                yield result
            else:
                continue

                
def create_merged_dataframe(csv_message,csv_categories):
    '''After extracting and cleaning the two csv file data,
    it creates single merged dataframe'''
    messages = pd.read_csv(csv_message)
    categories = pd.DataFrame(read_categories(csv_categories)
                              ,columns=columns)
    #removing duplicates
    messages=messages[~messages.duplicated()]
    categories=categories[~categories.duplicated()]
    df = messages.merge(categories, on = 'id', how='left')
    return df

def save_to_db(df,fileName, tableName):
    engine = create_engine(f'sqlite:///{fileName}')
    conn=engine.connect()
    engine.execute(f"DROP TABLE IF EXISTS {tableName}")
    df.to_sql(tableName, conn, index=False)
    return True

    
if __name__ =="__main__":
    _,messages,categories,db=get_file_names()
    df=create_merged_dataframe(messages,categories)
    tableName='categorizedMessages'
    a=save_to_db(df,db,tableName=tableName)
    if a : 
        print(f"Data has been saved into the {db} within \
the table '{tableName} '")

    
    
    
    

Overwriting etl_pipeline.py


In [2]:
#testing the file 
from etl_pipeline import get_file_names,read_categories,create_merged_dataframe,save_to_db

In [3]:
df=create_merged_dataframe('messages.csv','categories.csv')

In [4]:
save_to_db(df,'dataMessages.db',"categorizedMessages")

True

In [6]:
from sqlalchemy import create_engine
import pandas as pd
engine = create_engine(f'sqlite:///dataMessages.db')
conn=engine.connect()
data=pd.read_sql('select * from categorizedMessages',engine)
conn.close()
data

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1


In [7]:
!python etl_pipeline.py messages.csv categories.csv LabelledMessages.db

Data has been saved into the dataMessages.db within the table 'categorizedMessage '
